In [14]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [15]:
df = pd.read_csv('../Datasets/nyserda-electric-vehicle-drive-clean-rebate-data-beginning-2017-1.csv')

In [16]:
df

,Data through Date,Submitted Date,Make,Model,County,ZIP,EV Type,Transaction Type,Annual GHG Emissions Reductions (MT CO2e),Annual Petroleum Reductions (gallons),Rebate Amount (USD)
0,02/29/2020,01/13/2018,Ford,Focus,Westchester,10598,BEV,Lease,2.76,592.89,1700
1,02/29/2020,09/25/2017,Ford,Focus,Oneida,13601,BEV,Lease,2.76,592.89,1700
2,02/29/2020,04/26/2018,Ford,Focus,Suffolk,11776,BEV,Lease,2.76,592.89,1700
3,02/29/2020,08/30/2017,Ford,Focus,Nassau,11756,BEV,Lease,2.76,592.89,1700
4,02/29/2020,05/29/2018,Ford,Focus,Onondaga,13039,BEV,Purchase,2.76,592.89,1700
...,...,...,...,...,...,...,...,...,...,...,...
23169,02/29/2020,12/27/2017,Volvo,XC60 T8,Onondaga,13039,PHEV,Purchase,0.40,200.58,500
23170,02/29/2020,05/29/2018,Volvo,XC60 T8,Nassau,11791,PHEV,Purchase,0.40,200.58,500
23171,02/29/2020,09/25/2017,Volvo,XC90 T8,Niagara,14094,PHEV,Purchase,0.68,231.77,500
23172,02/29/2020,11/01/2017,Volvo,XC90 T8,Erie,14051,PHEV,Lease,0.68,231.77,500


In [17]:
df = df.drop(columns=["Data through Date", "ZIP"])

In [18]:
df = df.rename(columns={'Annual GHG Emissions Reductions (MT CO2e)': 'Annual CO2 Reductions', 'Annual Petroleum Reductions (gallons)': 'Annual Petroleum Reductions'})

In [19]:
df

,Submitted Date,Make,Model,County,EV Type,Transaction Type,Annual CO2 Reductions,Annual Petroleum Reductions,Rebate Amount (USD)
0,01/13/2018,Ford,Focus,Westchester,BEV,Lease,2.76,592.89,1700
1,09/25/2017,Ford,Focus,Oneida,BEV,Lease,2.76,592.89,1700
2,04/26/2018,Ford,Focus,Suffolk,BEV,Lease,2.76,592.89,1700
3,08/30/2017,Ford,Focus,Nassau,BEV,Lease,2.76,592.89,1700
4,05/29/2018,Ford,Focus,Onondaga,BEV,Purchase,2.76,592.89,1700
...,...,...,...,...,...,...,...,...,...
23169,12/27/2017,Volvo,XC60 T8,Onondaga,PHEV,Purchase,0.40,200.58,500
23170,05/29/2018,Volvo,XC60 T8,Nassau,PHEV,Purchase,0.40,200.58,500
23171,09/25/2017,Volvo,XC90 T8,Niagara,PHEV,Purchase,0.68,231.77,500
23172,11/01/2017,Volvo,XC90 T8,Erie,PHEV,Lease,0.68,231.77,500


Formatting Date Column

In [20]:
df[['Submitted Month', 'Submitted Year']] = df["Submitted Date"].apply(lambda x: pd.Series([x.split('/')[0], x.split('/')[2]]))

df = df.drop(columns=["Submitted Date", "Submitted Month"])

Label Encoding Categorical Columns

In [21]:
cols = ["Make", "Model", "EV Type","County","Transaction Type", "Rebate Amount (USD)", "Annual CO2 Reductions",	"Annual Petroleum Reductions"]

def label_encode(columns_to_encode):
    for column in columns_to_encode:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])

    return df

df = label_encode(cols)

In [23]:
df.to_csv('../Cleaned Datasets/rebate_cleaned.csv', index=False)